In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle

In [2]:
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None):
        self.data_list = data_list
        super().__init__(root, transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'

    def process(self):
        torch.save(self.collate(self.data_list), self.processed_paths[0])

In [3]:
mydataset = MyDataset('./data',None)

In [2]:
 
with open("graphs_data", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [3]:
read_graph_data[1]

Data(x=[8409, 1], edge_index=[2, 10956], edge_attr=[10956, 5], y=[1])

In [4]:
train_loader = DataLoader(read_graph_data, batch_size=32,shuffle=True)
test_loader = DataLoader(read_graph_data, batch_size=32,shuffle=False)

In [5]:
print(f'Dataset: {mydataset}:')
print('====================')
print(f'Number of graphs: {len(mydataset)}')
print(f'Number of node features: {mydataset.num_features}')
print(f'Number of edge features: {mydataset.num_edge_features}')
print(f'Number of classes: {mydataset.num_classes}')

Dataset: MyDataset(924):
Number of graphs: 924
Number of node features: 1
Number of edge features: 5
Number of classes: 2


In [6]:
train_loader = DataLoader(mydataset, batch_size=32, shuffle=True)
test_loader = DataLoader(mydataset, batch_size=32, shuffle=False)

In [7]:
from typing import Tuple, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import BatchNorm1d, Linear

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj, OptTensor, PairTensor


class CGConv(MessagePassing):

    def __init__(self, channels: Union[int, Tuple[int, int]], dim: int = 0,
                 aggr: str = 'add', batch_norm: bool = False,
                 bias: bool = True, **kwargs):
        super().__init__(aggr=aggr, **kwargs)
        # print(channels)
        self.channels = channels
        self.dim = dim
        self.batch_norm = batch_norm

        if isinstance(channels, int):
            channels = (channels, channels)
        self.lin_f = Linear(2*channels[0] + dim, channels[1], bias=bias)
        # self.lin_s = Linear(2*channels[0] + dim, channels[1], bias=bias)
        if batch_norm:
            self.bn = BatchNorm1d(channels[1])
        else:
            self.bn = None

        self.reset_parameters()


    def reset_parameters(self):
        self.lin_f.reset_parameters()
        # self.lin_s.reset_parameters()
        if self.bn is not None:
            self.bn.reset_parameters()


    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: PairTensor = (x, x)

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        out = out if self.bn is None else self.bn(out)
        out += x[1]
        # print(out.shape)
        # print(x[1])
        return out


    def message(self, x_i, x_j, edge_attr: OptTensor) -> Tensor:
        if edge_attr is None:
            z = torch.cat([x_i, x_j], dim=-1)
        else:
            z = torch.cat([x_i, x_j, edge_attr], dim=-1)

        # print(z.shape)
        # print(self.lin_f)
        return self.lin_f(z)

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.channels}, dim={self.dim})'

In [5]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = CGConv((mydataset.num_node_features, hidden_channels),mydataset.num_edge_features,batch_norm=True)
        self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [6]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
count = 0
def train():
    model.train()
    count = 0
    for data in train_loader:  # Iterate in batches over the training dataset.

        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        
        
        if data.y.shape[0] != out.shape[0]:
#             print(f'{out.shape} {data.y.shape} {data.batch.shape} ')
            continue

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss.cpu().item()
            
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.

        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        
#         print(f'{pred.shape} {data.y.shape} {data.batch.shape} ')
        if data.y.shape != pred.shape:
            continue

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.
# train_acc = test(test_loader)
# print(train_acc)

In [8]:

for epoch in range(1, 200):
    loss = train()
#     break
    train_acc = test(test_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, loss : {loss:.4f}')

Epoch: 001, Train Acc: 0.5433, loss : 0.6857
Epoch: 002, Train Acc: 0.5433, loss : 0.5289
Epoch: 003, Train Acc: 0.5433, loss : 0.4456
Epoch: 004, Train Acc: 0.5433, loss : 0.4879
Epoch: 005, Train Acc: 0.5433, loss : 0.5635


KeyboardInterrupt: 

In [ ]:
total = 0
for data in test_loader:  # Iterate in batches over the training dataset.
#     print(data.batch.max())
    total += data.y.sum()
#     break


In [ ]:
print(len(mydataset))
print(total)
total/len(mydataset)

In [ ]:
total = 0
for data in mydataset:
#     print(data.y)
    total += data.y
#     break
print(total)

In [ ]:
for i in range(100):  # Iterate in batches over the training dataset.
        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
#         print(data.shape)
        out = model(data.x, data.edge_index,data.edge_attr, data.batch)  # Perform a single forward pass.
#         print(out.shape)
#         break
        if out.shape[0] != data.y.shape[0]:
#             print((out.shape,data.y.shape))
            continue
#         print((out.shape[0],data.y.shape[0]))
#         break
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

In [ ]:
pred = out.argmax(dim=1)

In [ ]:
pred == data.y

In [ ]:
data.edge_attr = data.edge_attr.float() 
# print(data.edge_attr.float().dtype)
print(data.edge_attr.dtype)

In [ ]:
data.edge_attr 

In [ ]:
data.batch.max()
